# How to use MCDM methods

In this example, we will see how MCDM methods available in DESDEO can be
utilized to solve multiobjective optimization problems. MCDM methods are
implemented in a functional way, which means their individual components must be
first combined. This might feel unnecessary at first, but in practice, it means
we are able to combine components of multiple methods to create new methods that
can best suit our needs and the needs of decision makers.

Since scalarization sits often in a central role in DESDEO, we will first see
examples of how it can be done. Then, we will see examples of the
Reference Point Method and Synchronous NIMBUS in action. Throughout this
example, we will be solving the [river pollution
problem](../../api/desdeo_problem/#desdeo.problem.testproblems.river_pollution_problem),
consisting of two continuous variables and four objective functions ($f_1, f_2,
f_3, f_4$ to be maximized). We choose this problem because
of its relative simplicity.

## Scalarization in DESDEO

Scalarization is the transformation of a multiobjective optimization problem
into a single-objective optimization one. In DESDEO, this happens by adding a
`ScalarizationFunction` to an instance of the `Problem` class. However, because
many scalarization functions require information on either the ideal or nadir
point, or both, of the problem being solved, we first need to compute these
before proceeding with scalarization.

### Computing the ideal point and (approximating) the nadir point

To compute the ideal point, and approximate the nadir point, of the rive pollution problem, 
we can utilize the payoff table method
([`payoff_table_method`](../../api/desdeo_tools/#desdeo.tools.utils.payoff_table_method)).
We begin by creating an instance of the river pollution problem and
passing it to the payoff table method:

In [1]:
from desdeo.problem.testproblems import river_pollution_problem
from desdeo.tools import PyomoIpoptSolver, payoff_table_method

problem = river_pollution_problem(five_objective_variant=False)  # we use the four objective variant

ideal, nadir = payoff_table_method(problem, solver=PyomoIpoptSolver)
print(f"{ideal=}")
print(f"{nadir=}")

ideal={'f_1': 6.34000002019196, 'f_2': 3.444871831295394, 'f_3': 7.499999999424789, 'f_4': -7.777578581169564e-10}
nadir={'f_1': 4.751000003065099, 'f_2': 2.853461538677662, 'f_3': 0.32110955171648925, 'f_4': -9.706668887709498}


While the computed `ideal` point will have its components at the true ideal
points, as the nadir point will generally be a very rough approximation of the
true nadir point due to the nature of the payoff table method. We also notice
that we passed the
[`PyomoIpoptSolver`](../../api/desdeo_tools/#desdeo.tools.pyomo_solver_interfaces.PyomoIpoptSolver)
solver to the payoff table method. This solver interfaces to the
[Ipopt](https://coin-or.github.io/Ipopt/) from the COIN-OR project through a
Pyomo model (https://www.pyomo.org/), hence the name. Ipopt is suitable for
non-linear optimization.

<div class="admonition note">
<p class="admonition-title">Choosing the right solver</p>
<p>
While it is not required to supply a solver to many of the functions
utilizing them, relying on the automatic inference of the correct type of solver
implemented in DESDEO can be unpredictable. This is because the correct type of
solver cannot be always inferred from the properties of the problem alone, since
this could be transformed in various ways (e.g., when scalarized). Thus, it is
recommended to always manually choose and supply an appropriate solver, when
possible. 
</p>
</div>

In practice, the nadir point can be either inquired from a decision maker
(domain expert), or it can be approximated by solving the problem utilizing an
evolutionary method, such as [NSGA3](../../howtoguides/ea_options), and then
reading the nadir point values of the approximated Pareto front. However, while
inaccurate, the payoff table method works out-of-the-box with almost any
problem, and it gives at least a first workable approximation fo the nadir
point. The ideal point computed by the payoff table method is often accurate
enough to get some preliminary results. A more accurate approximation should be
considered for more ambitious tasks.

<div class="admonition note">
<p class="admonition-title">The payoff table method is not very accurate</p>
<p>The payoff table method is a very rough
way to approximate the nadir point of a problem. It is better to use an
evolutionary method instead to approximate the problem's Pareto front, and then
read an approximated value for the nadir point from the front.
</p>
</div>

Regardless the source of the information about the nadir (and ideal) point values,
we can update the problem with this information. For the purpose of this example,
the nadir point approximated by the payoff table method is accurate enough.

In [2]:
problem = problem.update_ideal_and_nadir(ideal, nadir)

print(f"Ideal: {problem.get_ideal_point()}")
print(f"Nadir: {problem.get_nadir_point()}")

Ideal: {'f_1': 6.34000002019196, 'f_2': 3.444871831295394, 'f_3': 7.499999999424789, 'f_4': -7.777578581169564e-10}
Nadir: {'f_1': 4.751000003065099, 'f_2': 2.853461538677662, 'f_3': 0.32110955171648925, 'f_4': -9.706668887709498}


It is important to note that the utilized method `update_ideal_and_nadir`
and then used its output to redefine the variable `problem`. In DESDEO, instances
of the `Problem` class are, in principle, __immutable__. This means that anytime
we wish to change an already instantiated `Problem` object, we essentially have
to create a new one. `Problem`s are chosen to be immutable in DESDEO to avoid
involuntarily changing the original problem, which can easily happen when
solving the problem with multiple methods.

<div class="admonition note">
<p class="admonition-title">Problems are immutable</p>
<p>Instances of the <code>Problem</code> class in DESDEO are immutable. When making
changes to an existing <code>Problem</code> objects, methods and functions will return a
new instance of the original <code>Problem</code> with the applied changes.</p>
</div>

### Scalarizing a problem

Once we have the ideal and nadir point values available, we can scalarize our
problem. We will use the achievement scalarizing function, or ASF. To utilize
the ASF, we will need a reference point. Let us use the values $\left[5.55,
3.20, 3.91, -4.85\right]$ for the reference point, as an example. Because the
river pollution problem is differentiable, we can use the differentiable variant of the
ASF ([`desdeo.tools.add_asf_diff`](../../api/desdeo_tools/#desdeo.tools.scalarization.add_asf_diff)):

In [3]:
from desdeo.tools import add_asf_diff

reference_point = {
    "f_1": 5.55,
    "f_2": 3.20,
    "f_3": 3.91,
    "f_4": -4.85,
}

# The first returned value is the problem with the scalarization, while the
# second is the symbol of the newly added scalarization function.
problem_w_asf, target = add_asf_diff(problem, symbol="asf", reference_point=reference_point)

When adding the ASF, we had to supply also a `symbol` in addition to the
`reference_point`. As discussed in the previous example, the `symbol` is
utilized to identify various components of a `Problem`. In this case, the
`symbol="asf"` is used to refer to the ASF, which was added to the problem.
Note that the `symbol` is just a name, it can be any string as long as the
instance of the `Problem` does not have existing components with the
same symbol.

We also mentioned that we can use the differentiable variant of the ASF, because
our problem is differentiable. This is important because we want to keep the
differentiability of our problem even after scalarization. This allows us to
use, e.g., gradient-based optimizers when solving the scalarized problem. In
turn, this allows us to get accurate and optimal solutions. If our problem was
not differentiable, then we could have used the non-differentiable variant of
ASF
([`desdeo.tools.add_asf_nondiff`](../../api/desdeo_tools/#desdeo.tools.scalarization.add_asf_nondiff).
This would be practical if we knew we had to solve our problem using
heuristics-based methods, which are impartial to the differentiability of our
problem. In fact, it could be even detrimental to utilize the differentiable
variant when solving the scalarized problem with an evolutionary method, since
the differentiable variant introduces additional constraints to the problem,
which evolutionary methods are not very adept to handle. The lesson here is,
that we should be knowledgeable enough about our problem to manipulate it in the
best possible way.

<div class="admonition note">
<p class="admonition-title">Custom scalarization functions</p>
<p>
While DESDEO provides functions (such as <code>add_asf_diff</code>) to add scalarization functions to problems,
user are by no means limited to only these. A scalarization function is nothing more than a field
in a <code>Problem</code> object. Users can define their own
<a href="../../api/desdeo_problem/#desdeo.problem.schema.ScalarizationFunction"><code>ScalarizationFunction</code> objects</a>
and add them to an instance of a <code>Problem</code> using its
<a href="../../api/desdeo_problem/#desdeo.problem.schema.Problem.add_scalarization"><code>add_scalarization</code></a>
method.
</p>
</div>

### Solving a scalarized problem

To solve the scalarized problem, we can once again utilize the `PyomoIpoptSolver`
interface. However, this time we will have to set it up ourselves instead of
just passing it as an argument to the payoff table method:

In [4]:
from pprint import pprint

In [5]:
solver = PyomoIpoptSolver(problem_w_asf)

result = solver.solve(target)

Notice how we passed the `target` to the `solve` method. This tells the solver, which
objective function should be optimized. The solver interfaces in DESDEO
return their optimization results in a Pydantic object. We can inspect its content
by printing it as a dictionary:

In [6]:
pprint(result.__dict__, width=120)

{'constraint_values': {'f_1_con': -0.3057121235731717,
                       'f_2_con': 5.254153390144012e-09,
                       'f_3_con': -9.922039873044852e-08,
                       'f_4_con': 4.461189334614701e-09},
 'extra_func_values': None,
 'message': "Pyomo solver status is: 'ok', with termination condition: 'optimal'.",
 'optimal_objectives': {'f_1': 6.209215255403704,
                        'f_2': 3.264552137173013,
                        'f_3': 4.693571830352962,
                        'f_4': -3.7905236281284678},
 'optimal_variables': {'_alpha': -0.1091493197477763, 'x_1': 0.9423855750677106, 'x_2': 0.9422934861100473},
 'scalarization_values': {'asf': -0.10915901059978278},
 'success': True}


Out of the results, we are likely most interested in the fields
`optimal_objectives` and `optimal_variables`, which represent the optimal
objective function and variable values for the found optimal solution of the
scalarized problem. The `success` field will indicate whether the optimization
was successful, and the `message` field will yield solver-specific information
on the success, failure, or other details, of the optimization process. Lastly, checking
the `constraint_values` field can be valuable as well, since as we recall from the
previous example, constraints in DESDEO are defined such that a negative value
means the constraint holds, and a positive that it is breached. Here we see
some values being positive, yet extremely close to zero. In practice, these
values can be safely assumed to be zero due to the nature and precision
of floating-point arithmetics on computers.

## Utilizing the Reference Point Method

The [Reference Point
method](../../api/desdeo_mcdm/#desdeo.mcdm.reference_point_method) is a
relatively simple interactive multiobjective optimization method, which utilizes
scalarization, and a reference point as preference information. The method uses
the reference point to create $k+1$, with $k$ being the number of objective
functions, sub-problems, which use a perturbed version of the original reference
point (one of the sub-problems uses the original unperturbed reference point as
well) to scalarize the problem being solved using the achievement scalarizing
function. The resulting solutions from the sub-problems will be more spread out
the farther the original reference point is from the true Pareto front of the
problem. This can allow a decision maker to get a more general picture of what
kind of solutions are available, when far from the front; and fine-tune their
solution once the decision maker begins to close on the Pareto front with the
given reference point. The method is interactive, which means that a decision
maker is expected to provide reference points iteratively during multiple
iterations of the method.

As the Reference Point Method is quite simple, we will need only one function to
operate it. We begin by importing it, we choose to use the same
reference point we utilized previously as the initial preference information, and then we
iterate the method:

In [7]:
from desdeo.mcdm import rpm_solve_solutions

results = rpm_solve_solutions(problem, reference_point, solver=PyomoIpoptSolver)

print(f"Number of solutions {len(results)}")
print(f"Original reference point: {reference_point}")
for i, result in enumerate(results):
    print(f"Solution {i + 1}: F={result.optimal_objectives}")

Number of solutions 5
Original reference point: {'f_1': 5.55, 'f_2': 3.2, 'f_3': 3.91, 'f_4': -4.85}
Solution 1: F={'f_1': 6.209215255403704, 'f_2': 3.264552137173013, 'f_3': 4.693571830352962, 'f_4': -3.7905236281284678}
Solution 2: F={'f_1': 6.333581100449999, 'f_2': 3.2320615106376724, 'f_3': 0.786903967588799, 'f_4': -3.086415992189174}
Solution 3: F={'f_1': 6.340000004252907, 'f_2': 3.444871833350942, 'f_3': 0.32111078266553417, 'f_4': -9.706668949083838}
Solution 4: F={'f_1': 6.035263422876624, 'f_2': 3.2609445613715065, 'f_3': 6.12463885963372, 'f_4': -3.8497337702333416}
Solution 5: F={'f_1': 6.240119482700062, 'f_2': 3.2220277112967315, 'f_3': 4.177385479588673, 'f_4': -3.0136070634586956}


As seen from the output, we got 5 solutions, which is expected with 4 objective
functions. Notice also how we passed the instance of the problem without an
added scalarization function (`problem`). The addition of the scalarization
function happened in the function implementing the Reference Point Method.

Suppose a decision maker now provides a new reference point: $[6.10, 3.50, 7.1, -5.67]$.
We use it to iterate the method again:

In [8]:
new_reference_point = {"f_1": 6.10, "f_2": 3.50, "f_3": 7.1, "f_4": -5.67}

results = rpm_solve_solutions(problem, new_reference_point, solver=PyomoIpoptSolver)

print(f"Number of solutions {len(results)}")
print(f"Original reference point: {new_reference_point}")
for i, result in enumerate(results):
    print(f"Solution {i + 1}: F={result.optimal_objectives}")

Number of solutions 5
Original reference point: {'f_1': 6.1, 'f_2': 3.5, 'f_3': 7.1, 'f_4': -5.67}
Solution 1: F={'f_1': 6.101764838919299, 'f_2': 3.3889712959545624, 'f_3': 5.752269857223239, 'f_4': -7.492283292660489}
Solution 2: F={'f_1': 6.340000018718053, 'f_2': 3.2332400604525633, 'f_3': 0.3211096655441512, 'f_4': -3.0982539086713006}
Solution 3: F={'f_1': 6.340000003499234, 'f_2': 3.44487183329319, 'f_3': 0.3211108408706522, 'f_4': -9.706668948836672}
Solution 4: F={'f_1': 5.5969121623973255, 'f_2': 3.3147371429058308, 'f_3': 7.096351295672935, 'f_4': -5.378453025905292}
Solution 5: F={'f_1': 6.162954687421054, 'f_2': 3.3476331401795956, 'f_3': 5.250483709221593, 'f_4': -5.901520917863092}


We notice how the solutions changed with the new reference point. This
process of providing a reference point and inspecting solutions can continue for
multiple iterations until a satisfactory solution is found, or the decision
maker gets tired, for instance. 


## Utilizing Synchronous NIMBUS

In the interactive [Synchronous NIMBUS (or just NIMBUS)
method](../../api/desdeo_mcdm/#desdeo.mcdm.nimbus), instead of providing a
reference point, a decision maker classifies each objective function value of a
Pareto optimal solution into one of five different classes in each iteration.
These classes are:

- $<$: to be improved from its current value,
- $\leq$: to be improved until some aspiration level is reached, 
- $\geq$: to be worsened until some reservation level is reached,
- $=$: to stay at its current value, and
- $0$: to change freely.

This classification information is then utilized to compute 1 to 4 (depending on
the wishes of the decision maker) new Pareto optimal solutions. Internally, for
each computed solution, a different scalarization function is formulated based
on the given classifications. The reason to utilize different scalarization
functions is that different functions will result in different Pareto optimal
solutions once solved, even if the same preference information is utilized.
Having different solution candidates available for the decision maker to inspect
can be argued to be beneficial from a decision-support perspective

Apart from a different preference type, in NIMBUS the decision maker has also
the possibility to save interesting solutions to an archive to be inspected
later, or chosen as the next solution to be classified. Furthermore, the
decision maker has also the option to request a desired number of solutions to
be computed between two previously found solutions. Thus, unlike in the Reference
Point Method, we will be utilizing more than just one function to build a simple
implementation of NIMBUS.

### Generating a starting point

As mentioned, in NIMBUS, the decision maker classifies the objective function
values of a Pareto optimal solution in each iteration. This means that we will
need an initial solution to begin with. This solution may come from outside the
method (e.g., from a previously utilized method), or it may be computed based on
an initial reference point given by a decision maker, for example; or we may
just utilize a "compromise" reference point, which is a reference point having
its aspiration level values sitting in the middle point of the objective
function's ideal and nadir point values, respectively.

If we keep
considering the river pollution problem, we have the previously computed solutions
available from the Reference Point Method. But in case we did not, we could use the
middle point values between the ideal and nadir values of each objective function
as a "compromise" reference point as follows:

In [9]:
# we recall that each objective function is to be maximized
compromise_reference_point = {obj.symbol: obj.nadir + (obj.ideal - obj.nadir) / 2.0 for obj in problem.objectives}

print(f"{compromise_reference_point=}")

compromise_reference_point={'f_1': 5.54550001162853, 'f_2': 3.149166684986528, 'f_3': 3.910554775570639, 'f_4': -4.853334444243628}


Then, to solve for the starting point utilizing a reference point, we utilize
the
[`generate_starting_point`](../../api/desdeo_mcdm/#desdeo.mcdm.nimbus.generate_starting_point)
function defined for NIMBUS:

In [10]:
from desdeo.mcdm.nimbus import generate_starting_point

result_starting_point = generate_starting_point(
    problem, reference_point=compromise_reference_point, solver=PyomoIpoptSolver
)

starting_point = result_starting_point.optimal_objectives

print(f"{starting_point=}")

starting_point={'f_1': 6.18458774309251, 'f_2': 3.24017387749916, 'f_3': 5.015253294101739, 'f_4': -3.3596585770809293}


### Classification of objective function values

Once we have a starting point, we can define the classifications of the
objective function values. In the implementation of NIMBUS, the classifications
are given as components of a reference point according to the following logic:

- $<$: for objective functions to be improved, we provide the objective function's ideal value;
- $\leq$: for objective functions to be improved until some aspiration level, we provide the aspiration level's value, which should be better than the current value of the objective function being classified, but not better than the ideal;
- $\geq$: for objective functions to be worsened until some reservation level, we provide the reservation level's value, which should be worse than the current value of the objective function being classified, but not worse than the nadir;
- $=$: for objective functions to be kept at their current value, we provide that objective function's current value; and
- $0$: for objective functions to change freely, we provide that objective function's nadir value.

In addition, the classifications given should overall allow at least one
objective function to be worsened and one to be improved. In other words, the
classifications should include at least one classification belonging to the
classes $<, \leq$, and one belonging to $\geq, 0$ to be valid.

Suppose now we consider the previously computed starting point $[6.18, 3.24,
5.02, -3.40]$ and we wish to improve the first two objective function values
(the first improve freely, and the second until the value $3.5$), keep the third
one as it is, and let the fourth one worsen until $-4.2$. The corresponding
reference point would then be as follows:

In [11]:
classification_reference_point = {
    "f_1": problem.get_objective("f_1").ideal,
    "f_2": 3.5,
    "f_3": starting_point["f_3"],
    "f_4": -4.2,
}

While the reference point defined above is enough to compute new solutions, we
can also utilize the function
[`infer_classifications`](api/desdeo_mcdm/#desdeo.mcdm.nimbus.infer_classifications)
to double-check our classifications:

In [12]:
from desdeo.mcdm.nimbus import infer_classifications

print(f"Inferred classifications: {infer_classifications(problem, starting_point, classification_reference_point)}")

Inferred classifications: {'f_1': ('<', None), 'f_2': ('<=', 3.5), 'f_3': ('=', None), 'f_4': ('>=', -4.2)}


In the above output, we see that the tuple corresponding to each objective
function's symbol contains the classification as its first element, and an
aspiration/reservation level as its second element. The `None` values are
present since the classifications $<, =$ (and $0$ as well) can be inferred from
the properties of the problem.

### Solving for new solutions

Once we have a starting point and a reference point corresponding to a valid classification,
we can solve for 1 to 4 new Pareto optimal solutions. For this, we invoke
[`solve_sub_problem`](../../api/desdeo_mcdm/#desdeo.mcdm.nimbus.solve_sub_problems) defined
for NIMBUS, and choose to solve for 3 new solutions:

In [13]:
from desdeo.mcdm.nimbus import solve_sub_problems

results = solve_sub_problems(
    problem,
    current_objectives=starting_point,
    reference_point=classification_reference_point,
    num_desired=3,
    solver=PyomoIpoptSolver,
)

print(f"Number of solutions {len(results)}")
print(f"Original reference point: {classification_reference_point}")
for i, result in enumerate(results):
    print(f"Solution {i + 1}: F={result.optimal_objectives}")

Number of solutions 3
Original reference point: {'f_1': 6.34000002019196, 'f_2': 3.5, 'f_3': 5.015253294101739, 'f_4': -4.2}
Solution 1: F={'f_1': 6.184587734651428, 'f_2': 3.2833345578487974, 'f_3': 5.0152533936919355, 'f_4': -4.200000009641636}
Solution 2: F={'f_1': 6.339998131048082, 'f_2': 3.4447027033431477, 'f_3': 0.3212554447205367, 'f_4': -9.696572712990864}
Solution 3: F={'f_1': 6.273460544173023, 'f_2': 3.3673524124620227, 'f_3': 3.4051007830045883, 'f_4': -6.377108082020183}


From the output of the above cell, we see that our starting point and provided
classifications have resulted in three new and clearly distinct (Pareto optimal)
solutions. These differ because, as mentioned, different scalarization functions
will generally lead to different solutions when the respective scalarized
problem is solved, even if using the same preference information.

Next, after inspecting the new solutions, the decision maker may choose one of
the solutions as the next solution to be classified (or even as the final one,
if they are already happy).  Alternatively, they may choose to store one or more
of the solutions to an archive, from which they may also choose the next
solution to be classified, if the archive had solutions saved to it prior to the
current iteration.  The decision maker may also choose to select two of any
previously computed solutions in the NIMBUS method, and ask for a desired number
of solutions to be generated between these two points.

Let us next consider a situation in which the decision maker would like to generate five
solutions between the starting point and the second solution computed ("Solution
2") based on the classifications provided in the first iteration. We use the
function
[`solve_intermediate_solutions`](api/desdeo_mcdm/#desdeo.mcdm.nimbus.solve_intermediate_solutions)
for this as demonstrated:

In [14]:
from desdeo.mcdm.nimbus import solve_intermediate_solutions

num_desired_intermediate = 5

intermediate_results = solve_intermediate_solutions(
    problem,
    solution_1=result_starting_point.optimal_variables,
    solution_2=results[1].optimal_variables,
    num_desired=num_desired_intermediate,
    solver=PyomoIpoptSolver,
)

print(f"Number of solutions {len(intermediate_results)}")
print(f"Original reference point: {classification_reference_point}")
for i, result in enumerate(intermediate_results):
    print(f"Solution {i + 1}: F={result.optimal_objectives}")

Number of solutions 5
Original reference point: {'f_1': 6.34000002019196, 'f_2': 3.5, 'f_3': 5.015253294101739, 'f_4': -4.2}
Solution 1: F={'f_1': 6.3177966405208394, 'f_2': 3.406465515147123, 'f_3': 1.7240127837232544, 'f_4': -7.80410553904878}
Solution 2: F={'f_1': 6.295595155964072, 'f_2': 3.371973817669948, 'f_3': 2.695032285288442, 'f_4': -6.49346935727902}
Solution 3: F={'f_1': 6.273393672030645, 'f_2': 3.3407008310438338, 'f_3': 3.4069597187102287, 'f_4': -5.532216370585998}
Solution 4: F={'f_1': 6.25119218807678, 'f_2': 3.3122142094511497, 'f_3': 3.951214791464647, 'f_4': -4.797169840121453}
Solution 5: F={'f_1': 6.228990704286701, 'f_2': 3.286155787541856, 'f_3': 4.3807327997279994, 'f_4': -4.216967645726693}


Notice that the function `solve_intermediate_solutions` requires the decision
variables of the solutions between which to compute the points. Internally, a
number of linearly spaced decision vectors, corresponding to the desired number
of new solutions, are generated between `solution_1` and `solution_2`.  These
vectors are then used to evaluate the problem being solved, and the result of
these evaluations are used as reference points used in a scalarization function
to solve for new Pareto optimal solutions.

<div class="admonition note">
    <p class="admonition-title">Solving intermediate solutions in NIMBUS</p>
    <p>
        According to how the Synchronous NIMBUS method has been defined in its
        original publication, the intermediate solutions are computed according
        to the decision vector values generated between the two supplied
        solutions. For problems with relatively linear objective functions and
        continuous variables, this can make sense.  However, in general, it
        might be a better idea to generate the points directly in the objective
        space of the problem between the objective function values of the
        provided solutions. For this, DESDEO provides the function
        <a
        href="../../api/desdeo_mcdm/#desdeo.mcdm.reference_point_method.rpm_intermediate_solutions"><code>rpm_intermediate_solutions</code></a>,
        which should be useful in any general case where such intermediate
        solutions are needed.  We have chosen to showcase the original way of
        computing the intermediate solutions to not deviate from the original
        publication.
    </p>
</div>

### Saving solutions

We have mentioned that the decision maker has the possibility to also save
solutions in the NIMBUS method.  In a notebook environment, this can mean
storing the solutions to, e.g., a `dict`. But in practice, this can be
cumbersome. This is why in NIMBUS, and many other interactive methods in DESDEO,
we have not implemented specific functions necessarily for all the features of
an interactive method, but have instead focused on implementing the features
that are algorithmic and mathematical in nature, which are all available
for the implemented methods.
It makes much more sense for the archiving of solutions in NIMBUS to be part of
the application implementing the user interface for the method. Therefore, these
features are being implemented in the web-API and web-GUI of DESDEO instead.

## Conclusions

We have seen two examples of how MCDM methods are implemented, and how they can
be operated, in a notebook environment in DESDEO.  The basic workflow consists
of calling various functions, inspecting their output, and possibly utilizing
the output in further calls to other functions. It is relatively easy to see how
these functions can be combined in various ways to not just re-create the
existing methods they are related to, but new ones as well.

We also noted that the input of a decision maker is central, and information
generated by the functions implementing components of MCDM methods is often to
be inspected by a decision maker. This makes a notebook interface very
impractical, calling for a dedicated interface instead. In practice, we would
use at least visualizations to better convey communicate information about the
solutions to a decision maker.  However, the purpose of this example has been to
showcase how the algorithmic aspect of interactive methods can be utilized in
DESDEO. Users are encouraged to utilize the user interfaces and visualizations
provided in the Web-GUI of the framework, or their own, in real-life
decision-support settings and studies involving human decision makers.